# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

TALKS = "talks_laura.tsv"

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat TALKS

cat: TALKS: No such file or directory


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv(TALKS, sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Hardware-effective Approaches for Skill Extrac...,Talk,sheffield,"NLP Seminars, University of Sheffield",2024-12-12,Online,NaN,This talk will present hardware-effective meth...
1,How AI is Shaping Your Job Hunt?,Panel,cinfo,Forum cInfo: The Swiss Event on Work and Caree...,2024-11-22,"Bern, Switzerland",https://www.cinfo.ch/en/news/forum-cinfo-2024-...,Panel discussion about AI's transformative rol...
2,AI-based Soft Skills Matching,Workshop,ehl,"Talent Management Systems Course, EHL Business...",2024-11-07,"Lausanne, Switzerland",NaN,Skill extraction introduction and annotations ...
3,Navigating Skills Detection in AI-Enhanced ATS,Talk,arca,Arca24 S.A,2024-09-10,"Novazzano, Switzerland",NaN,SEM24 Sharing Session for All Arca24 Employees.
4,Navigating Skills Detection and Bias in AI-Enh...,Talk,hesso,9th AI Valais/Wallis Workshop on Human-AI Team...,2024-04-26,"Sierre, Switzerland",https://www.hevs.ch/en/events/9th-ai-valais-wa...,"In recent years, the automation of skills dete..."
5,The use of AI in Talent Management Systems,Talk,gap,"Online, Growth Acceleration Partners (GAP)",2024-03-14,Costa Rica,https://www.youtube.com/watch?v=YNPvWtKslm4&fe...,"In this talk, we will learn about the applicat..."
6,Language technologies for industry: Research i...,Talk,ehl,"Talent Management Systems Course, EHL Business...",2023-11-16,"Lausanne, Switzerland",NaN,Skill extraction introduction and annotations ...
7,Text Simplification Open Discussion Session,Chair,tsar,14th Conference in Recent Advances in Natural ...,2023-09-08,"Varna, Bulgaria",https://aclanthology.org/volumes/2023.tsar-1/,Moderator for the Text Simplification future d...
8,Text Simplification at MilaNLP,Talk,milanlp,The Milan Natural Language Processing Group,2023-05-26,Online,NaN,Introduction to Text Simplification and Annota...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2012-03-01-talk-1.md     2023-09-08-tsar.md       2024-11-07-ehl.md
2013-03-01-tutorial-1.md 2023-11-16-ehl.md        2024-11-22-cinfo.md
2014-02-01-talk-2.md     2024-03-14-gap.md        2024-12-12-sheffield.md
2014-03-01-talk-3.md     2024-04-26-hesso.md
2023-05-26-milanlp.md    2024-09-10-arca.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
